1. How many trades were done (segmented into long/short)?

In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
#Q1
#How many trades were done (segmented into long/short)?
data= pd.read_csv("Sample PNL (Jan-Feb2021 - Global).csv")
data_short = data[data.Side == "short"]
data_long = data[data.Side == "long"]
print("number of short trades: ", data_short.Side.count())
print("number of long trades: ",data_long.Side.count())
print("total trades: ", data.Side.count())
print(data)

number of short trades:  139
number of long trades:  256
total trades:  395
             Ticker   ATR       Reason   Side  ATR(x)  ATR Used  Risk  \
0     TAN US Equity  5.01          AH+   long     1.0      5.01  2000   
1    ARKK US Equity  4.83       reduce   long     2.0      9.66  2000   
2    GBTC US Equity  0.86        HEDGE   long     1.5      1.29  2000   
3     NIO US Equity  2.93          AH+   long     2.5      7.33  2000   
4    QBTC CN Equity  2.52      reduced   long     1.0      2.52  9000   
..              ...   ...          ...    ...     ...       ...   ...   
390  VCNX US Equity  0.34  partnership   long     3.0      1.03  1200   
391  RETO US Equity  0.16      no news   long     3.0      0.49  1200   
392  CASA US Equity  0.46      results  short     2.0      0.92  1200   
393   JMP US Equity  0.33      results  short     2.0      0.66  1200   
394  IMNM US Equity  2.15       Covid+  short     3.0      6.46  1200   

     Position  Position(R)  Entry Price  ...  D

2. What was the hit ratio ( segmented into long/short )?  Hit ratio is defined as the percentage of winning trades out of all trades. 

In [2]:
#Q2
#What was the hit ratio ( segmented into long/short )?  Hit ratio is defined as the percentage of winning trades out of all trades.
trades=data.Side.count()
trades_short= data_short.Side.count()
trades_long= data_long.Side.count()
rg_short=data_short[data_short["Realized gains"] >=0].Side.count()
rg_long=data_long[data_long["Realized gains"] >=0].Side.count()

print("hit ratio for short trades: ", "{:.2%}".format( rg_short / trades_short ))
print("hit ratio for long trades: ", "{:.2%}".format( rg_long / trades_long ))
print("total trades: ", "{:.2%}".format( (rg_short + rg_long) / trades ))

hit ratio for short trades:  47.48%
hit ratio for long trades:  44.92%
total trades:  45.82%


In [3]:
data_short = data[data.Side == "short"]
data_long = data[data.Side == "long"]

print("number of short trades: ", data_short["Position"].sum())
print("number of long trades: ",data_long["Position"].sum())
print("total trades: ", data.Position.sum())

number of short trades:  1129707.81
number of long trades:  1499049.71
total trades:  2628757.52


3. Provide a list of markets I traded in, with specific no. of trades per market, and individual market performance of both long and short (a dictionary with country code for tickers is attached herein - UV = US but traded OTC)

In [4]:
#Q3
#Provide a list of markets I traded in, with specific no. of trades per market, and individual market performance of both 
#long and short (a dictionary with country code for tickers is attached herein - UV = US but traded OTC)
dict1 = {
"HK Equity" : "Hong Kong   ",
"SW Equity" : "Switzerland ",
"GY Equity" : " Germany    ",
"FP Equity" : "France      ",
"IM Equity" : "Italy       ",
"NA Equity" : "Netherlands ",
"SM Equity" : "Spain       ",
"FH Equity" : "Finland     ",
"SS Equity" : "Sweden      ",
"DC Equity" : "Denmark     ",
"NO Equity" : "Norway      ",
"LN Equity" : "London      ",
"BB Equity" : "Belgium     ",
"AV Equity" : "Austria     ",
"PL Equity" : "Portugal    ",
"JT Equity" : "Japan       ",
"US Equity" : "USA         ",
"CN Equity" : "Canada      "
}
#remove white spaces to both keys and values for below dictionary
dict1 =dict((k.strip(), v.strip()) for k, v in dict1.items())
dict1["UV Equity"]="USA OTC"
dict1_key = list(dict1)

data_new=data.copy()
#use dictionary to add a new column which contains string of key in dictionary to ticker column
data_new['market'] = data_new['Ticker'].apply(lambda x: "|".join( (v for (k,v) in dict1.items() if k in x ) ) )
#df1=df[["Ticker","market"]]
#check UV equity
#df1[df1.Ticker.str.contains("UV Equity")]


#b=data_new[["Ticker","market"]].loc[data_new.Ticker =="HIF1 Index"]
#b.apply(lambda x: x.str.strip())
#(data_new.market.str.len()==0).any()
data_new.loc[data_new["Realized gains"] >=0, 'winning trade'] = "win"
data_new.loc[data_new["Realized gains"] <0, 'winning trade'] = "lose"
market=data_new.groupby(['market', 'Side','winning trade']).agg({'winning trade': 'count'})
market["hit ratio"] = market["winning trade"] / market.groupby(['market', 'Side'])["winning trade"].transform("sum")
market1= market.groupby(['market', 'Side']).first()
print(market1)
print(data_new[["Entry Date", "Exit Date"]])

                   winning trade  hit ratio
market      Side                           
            short              3   0.600000
Belgium     short              3   1.000000
Canada      long               3   0.375000
            short              2   1.000000
Denmark     long               3   1.000000
            short              2   0.500000
Finland     short              1   1.000000
France      long               6   0.461538
            short              1   0.250000
Germany     long              13   0.406250
            short              8   0.571429
Hong Kong   long              35   0.573770
            short             13   0.590909
Italy       long               3   0.500000
London      long               9   0.750000
            short              2   0.666667
Netherlands long               1   0.200000
            short              1   1.000000
Norway      long               2   0.666667
Portugal    long               1   1.000000
Spain       long               2

4. What was the average holding period of my positions? Please segment this into individual markets. Trades with indexes should be omitted here (for instance, average holding period in Denmark was 2 weeks; average holding period in HK was 2 days, etc.)

In [5]:
#Q4
#What was the average holding period of my positions? Please segment this into individual markets. 
#Trades with indexes should be omitted here
#(for instance, average holding period in Denmark was 2 weeks; average holding period in HK was 2 days, etc.)
import datetime
from datetime import timedelta
data_q4 = data_new.copy()
data_q4 = data_q4[["Ticker","Entry Date", "Exit Date", "market","Day 2 Date"]]

In [6]:
### handle entry date ###
#count number of strings for each row and see the difference of date format
print("data_q4 \n " ,  data_q4["Entry Date"].str.len().value_counts())

#add missing year to the date format in Exit Date column
new_entry_date=[]
for i, j in enumerate(data_q4["Entry Date"]):
    if len(j)!=10:
        new_entry_date.append(pd.to_datetime(data_q4["Day 2 Date"][0]) - timedelta(days=1))
    else:
        new_entry_date.append(j)
data_q4_new = data_q4.copy()
data_q4_new["Entry Date"]=new_entry_date
data_q4_new["Entry Date"] = pd.to_datetime(data_q4_new["Entry Date"])
#data_q4["Entry Date"]= (pd.to_datetime(data_q4["Day 2 Date"]) - timedelta(days=1))
data_q4_new

data_q4 
  10    302
5      85
8       4
9       4
Name: Entry Date, dtype: int64


,Ticker,Entry Date,Exit Date,market,Day 2 Date
0,TAN US Equity,2021-01-07,01/11/2021,USA,1/8/2021
1,ARKK US Equity,2021-01-07,01/11/2021,USA,1/8/2021
2,GBTC US Equity,2021-01-07,01/11/2021,USA,11/6/2020
3,NIO US Equity,2021-01-07,01/11/2021,USA,1/5/2021
4,QBTC CN Equity,2020-11-17,01/11/2021,Canada,11/18/2020
...,...,...,...,...,...
390,VCNX US Equity,2021-02-19,02/19/2021,USA,2/22/2021
391,RETO US Equity,2021-02-22,02/22/2021,USA,2/23/2021
392,CASA US Equity,2021-02-19,02/19/2021,USA,2/22/2021
393,JMP US Equity,2021-02-19,02/19/2021,USA,2/22/2021


In [7]:
### handle exit date ###
#count number of strings for each row and see the difference of date format
print("data_q4 \n " ,  data_q4["Exit Date"].str.len().value_counts())

#add missing year to the date format in Exit Date column
new_exit_date=[]
for i, j in enumerate(data_q4["Exit Date"]):
    if len(j)!=10:
        new_exit_date.append(j+"/"+ str(pd.DatetimeIndex(pd.to_datetime(data_q4["Day 2 Date"])).year[i]))
    else:
        new_exit_date.append(j)
#cast to pandas series    
new_exit_date= pd.Series(new_exit_date)
#new_exit_date=pd.to_datetime(new_exit_date) 
data_q4_new["Exit Date"]=new_exit_date
print("data_q4_new \n ", data_q4_new["Exit Date"].str.len().value_counts())
data_q4_new["Exit Date"] = pd.to_datetime(data_q4_new["Exit Date"])
data_q4_new["Day 2 Date"] = pd.to_datetime(data_q4_new["Day 2 Date"])
data_q4_new

data_q4 
  10    310
5      85
Name: Exit Date, dtype: int64
data_q4_new 
  10    395
Name: Exit Date, dtype: int64


,Ticker,Entry Date,Exit Date,market,Day 2 Date
0,TAN US Equity,2021-01-07,2021-01-11,USA,2021-01-08
1,ARKK US Equity,2021-01-07,2021-01-11,USA,2021-01-08
2,GBTC US Equity,2021-01-07,2021-01-11,USA,2020-11-06
3,NIO US Equity,2021-01-07,2021-01-11,USA,2021-01-05
4,QBTC CN Equity,2020-11-17,2021-01-11,Canada,2020-11-18
...,...,...,...,...,...
390,VCNX US Equity,2021-02-19,2021-02-19,USA,2021-02-22
391,RETO US Equity,2021-02-22,2021-02-22,USA,2021-02-23
392,CASA US Equity,2021-02-19,2021-02-19,USA,2021-02-22
393,JMP US Equity,2021-02-19,2021-02-19,USA,2021-02-22


In [8]:
### holding period ###
# as type of date calculation is timedelta, so we use dt.day to cast int
data_q4_new["holding period"]=(data_q4_new["Exit Date"] - data_q4_new["Entry Date"]).dt.days
# average holding period
data_q4_new.groupby("market").mean()

,holding period
market,
,10.200000
Belgium,3.000000
Canada,21.900000
Denmark,2.714286
Finland,0.000000
France,4.000000
Germany,5.021739
Hong Kong,10.698795
Italy,2.500000


In [9]:
#fix
data_q4_new[data_q4_new["holding period"]<0]

,Ticker,Entry Date,Exit Date,market,Day 2 Date,holding period
30,6123 HK Equity,2021-01-07,2021-01-05,Hong Kong,2021-01-06,-2


data_q4[["Exit Date","year"]]